# 1) DeepLabv2 tests

In [1]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [2]:
import numpy as np
import pandas as pd
import cv2
import yaml
import torch
import torch.nn.functional as F
from torch import nn
from models.deeplabv2 import DeepLabV2
from models.msc import MSC
from addict import Dict

In [3]:
def DeepLabV2_ResNet101_MSC(n_classes):
    return MSC(
        base=DeepLabV2(n_classes=n_classes, 
                       n_blocks=[3, 4, 23, 3], 
                       atrous_rates=[6, 12, 18, 24]),
        scales=[0.5, 0.75])

In [4]:
def get_device(use_cuda=True):
    cuda = use_cuda and torch.cuda.is_available()
    device = torch.device("cuda" if cuda else "cpu")
    if cuda:
        current_device = torch.cuda.current_device()
        print("Device:", torch.cuda.get_device_name(current_device))
    else:
        print("Device: CPU")
    return device

In [5]:
def setup_model(device, model_path, n_classes, train=True):
    model = DeepLabV2_ResNet101_MSC(n_classes=n_classes)
    # if using last version of pytorch, can load only in GPU
    state_dict = torch.load(model_path, map_location=lambda storage, loc: storage) 
    if train:
        model.load_state_dict(state_dict, strict=False)  # to skip ASPP
        model = nn.DataParallel(model)
    else:
        model.load_state_dict(state_dict)
        model = nn.DataParallel(model)
        model.eval()
    model.to(device)
    return model

In [6]:
_config = Dict(yaml.load(open('../config/cocostuff164k.yaml')))

In [7]:
device = get_device(use_cuda=True)
model = setup_model(device, 
                    '../models_parameters/deeplab_orig_cocostuff164k_iter100k.pth', 
                    182, train=False)

Device: GeForce GTX 1050 Ti with Max-Q Design


In [8]:
def _add_merged_stuff(_classes):
        # This model was not trained with the new stuff-merged, so map manually
        # using this string from http://cocodataset.org/#panoptic-eval
        # Also set to "VOID" (-1) deleted stuff.
        s = """\
        tree-merged: branch, tree, bush, leaves
        fence-merged: cage, fence, railing
        ceiling-merged: ceiling-tile, ceiling-other
        sky-other-merged: clouds, sky-other, fog
        cabinet-merged: cupboard, cabinet
        table-merged: desk-stuff, table
        floor-other-merged: floor-marble, floor-other, floor-tile
        pavement-merged: floor-stone, pavement
        mountain-merged: hill, mountain
        grass-merged: moss, grass, straw
        dirt-merged: mud, dirt
        paper-merged: napkin, paper
        food-other-merged: salad, vegetable, food-other
        building-other-merged: skyscraper, building-other
        rock-merged: stone, rock
        wall-other-merged: wall-other, wall-concrete, wall-panel
        rug-merged: mat, rug, carpet"""
        # Turn string into useful mapping
        map_into_merged_int = {vv: idx+183 for idx, (k, v) in enumerate(
            x.split(": ") for x in s.split("\n")) for vv in v.split(", ")}
        # Add mapping for delete stuff
        map_into_merged_int.update({k: -1 for k in [
            "furniture-other", "metal", "plastic", "solid-other",
            "structural-other", "waterdrops", "textile-other", "cloth",
            "clothes", "plant-other", "wood", "ground-other"]})

        _inv = {v: k for k, v in _classes.items()}
        _map_to_merged = {_inv[k]: v for k, v in map_into_merged_int.items()}

        extend_stuff_merged = {idx+183: k for idx, (k, v) in enumerate(
            x.split(": ") for x in s.split("\n"))}
        _classes.update(extend_stuff_merged)
        _classes.update({-1: "VOID"})
        
        return _map_to_merged

In [9]:
_classes = {}
with open('../data/labels_2.txt') as f:
    for label in f:
        label = label.rstrip().split("\t")
        _classes[int(label[0])] = label[1].split(",")[0]
_map_to_merged = _add_merged_stuff(_classes)

In [10]:
def _replace_labels_with_merged(_map_to_merged, labelmap):
    # Simpler, just use pandas
    return pd.DataFrame(labelmap).replace(_map_to_merged).values

In [11]:
def _preprocess_one(img):
    image = img.copy().astype(float)
    scale = _config.IMAGE.SIZE.TEST / max(image.shape[:2])
    image = cv2.resize(image, dsize=None, fx=scale, fy=scale)
    image -= np.array(
        [
            float(_config.IMAGE.MEAN.B),
            float(_config.IMAGE.MEAN.G),
            float(_config.IMAGE.MEAN.R),
        ]
    )
    return image.transpose(2, 0, 1)

In [12]:
def _preprocess_image(device, imgs):
    buff = []
    for img in imgs:
        buff.append(_preprocess_one(img))
    image = torch.from_numpy(np.array(buff)).float()
    return image.to(device)

In [13]:
def predict(device, model, img):
    """Predict on one image or batch
    Return:
        labelmap, labels
    """
    if isinstance(img, np.ndarray) and img.ndim == 3:
        return _predict_batch(device, model, [img])
    return _predict_batch(device, model, img)

In [14]:
def _predict_batch(device, model, imgs):
    # Note: it surprisingly does not speedup to run on bacthes
    # TODO: check pytorch implem deeper and find why
    image = _preprocess_image(device, imgs)
    model.to(device)
    output = model(image)
    # 0.2s
    output = F.interpolate(
        output,
        size=imgs[0].shape[:2],
        mode="bilinear", align_corners=True
    )
    output = F.softmax(output, dim=1)
    output = output.data.cpu().numpy()

    labelmaps = np.argmax(output, axis=1)
    labelmaps = np.array([
        _replace_labels_with_merged(_map_to_merged, x) for x in labelmaps])
    labels = np.array([np.unique(l) for l in labelmaps])
    return labelmaps, labels

In [ ]:
img = cv2.imread('../data/test/000000000885.jpg')
device = get_device(use_cuda=False)
predict(device, model, img)

It works as expected, except that I cannot use it on a cpu because the pre-trained model was trained on a gpu on a different version of pytorch. I'll handle this later.   
The next step is to implement this properly. Then I'll build the instance segmentation part.